<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/the_state_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
# Import packages

pip install streamlit
pip install streamlit-chat

import os
import json
import streamlit as st
from streamlit_chat import message

In [ ]:
# Global parameters

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
LOAD_FILE = os.path.join(LOAD_PATH, "guideline_structured.json")

SAVE_PATH = "DEFINE"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "DEFINE")

PATIENT_DICTIONARY = {}

In [ ]:
# Load LLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [ ]:
# Load JSON

def load_json():
    try:
        with open(LOAD_FILE, "r", encoding="utf-8") as guideline_structured:
            return json.load(guideline_structured)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {LOAD_FILE}')

guideline_structured = load_json()

print(type(guideline_structured))
print(len(guideline_structured))
print(guideline_structured[0])

In [ ]:
# Create record of exact guideline version extracted with metadata
"""In case of future runs with differernt guidelines and structure
Then problems with a run can be traced precisely"""
- guideline name
- html
- scrape date

In [ ]:
# Save local copy of JSON with metadata
- protect the file somehow from being overwritten or edited?

In [ ]:
# Create an index or summary for the JSON to enable more reliable searching by the state manager?

In [ ]:
#### NOTE
"""
If the question being asked tessellates with a particular section (e.g., a medication)
it may be worth flagging other areas (e.g., carer support.. physical health...)
The model may need to supply back to the user a list of "have you also considered..." making use of these headings
to prevent a narrow focus on the main content of the question
and ensure that other recommendation sections are also being considered
"""

In [ ]:
def get_user_message():
    """
    Uses a UI like Streamlit or an API to send and receive messages
    """

    # From https://www.geeksforgeeks.org/python/create-a-chatbot-with-openai-and-streamlit-in-python/
    st.title(&quot;NICE GUIDELINE CHATBOT&quot;)
    if 'user_input' not in st.session_state:
        st.session_state['user_input'] = []

    if 'openai_response' not in st.session_state:
        st.session_state['openai_response'] = []

    def get_text():
        input_text = st.text_input(&quot;Enter your question and patient information here:&quot;, key=&quot;input&quot;)
        return input_text

    user_input = get_text()

    if user_input:
        output = api_calling(user_input)
        output = output.lstrip(&quot;\n&quot;)

        # Store the output
        st.session_state.openai_response.append(user_input)
        st.session_state.user_input.append(output)


In [ ]:
def call_llm(prompt):
    """
    General function for any call to the LLM
    What is passed to the LLM ("prompt") should be decided by other functions
    """

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

In [ ]:
def extract_patient_facts(user_input):
    """
    Use LLM to decompose message into fields captured by JSON-keys
    """

    user_input = "USER INPUT: " + (None)

    prompt = """You are extracting structured information from USER INPUT.

                RULES:
                - 'age' should be an integer
                - 'gender' should be 'male', 'female' or 'other'
                - 'medications' should be a dictionary of medicaitons limited to known medication names, with duration taken also recorded
                - 'recent_medical_events'
                - 'patient_preferences'

                Produce JSON with exactly these keys:
                - age
                - gender
                - medications
                - recent_medical_events
                - patient_preferences
                """

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    patient_facts = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return patient_facts[0]

In [ ]:
def select_relevant_recommendations(patient_facts):
    """
    Based on matches between decomposed user input and guideline_structured.json
    """

    matching_fields = [] # A data object capturing all fields shared between decomposed user input and the structured guideline

    relevant_recommendations = [] # A place to store the full text of original recommendations

    for i, v in patient_facts: # Iterate over each key-value pair in patient_facts
        if v in guideline_structured: # Check whether the value appears in the json
            matching_fields.append(i, v) # Add that value to "matching_fields"



In [ ]:
def decide_next_question_or_answer():
    """
    - must record history of decisions taken for audit
    """


In [ ]:
def generate_question():
    # Needed as separate step?

In [ ]:
def update_patient_dict():
    """
    - what is known about the "patient" object
    - history of questions asked and user answers, or is this in the UI?
    """